In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

from desc.plotting import *

In [2]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.io import netcdf_file
from scipy.optimize import root

from simsopt.field.boozermagneticfield import (
    BoozerRadialInterpolant,
    InterpolatedBoozerField,
)
import warnings

In [3]:
def boozer_to_cylindrical(field, s, theta, zeta):
    r"""
    Convert from Boozer coordinates to cylindrical coordinates.

    Args:
        field : The :class:`BoozerMagneticField` instance used for field evaluation.
        s : A scalar or a numpy array of shape (npoints,) containing the
            normalized toroidal flux.
        theta : A scalar or a numpy array of shape (npoints,) containing the
            Boozer poloidal angle.
        zeta : A scalar or a numpy array of shape (npoints,) containing the
            Boozer toroidal angle.

    Returns:
        R : A scalar or a numpy array of shape (npoints,) containing the
            radial coordinate.
        phi : A scalar or a numpy array of shape (npoints,) containing the
            azimuthal angle.
        Z : A scalar or a numpy array of shape (npoints,) containing the
            vertical coordinate.
    """
    if not isinstance(s, np.ndarray):
        s = np.asarray(s)
    if not isinstance(theta, np.ndarray):
        theta = np.asarray(theta)
    if not isinstance(zeta, np.ndarray):
        zeta = np.asarray(zeta)

    # Handle scalar inputs - return scalars if any input is a scalar
    input_scalar = np.isscalar(s) or np.isscalar(theta) or np.isscalar(zeta)

    # Ensure all arrays have the same shape
    if s.shape != theta.shape or s.shape != zeta.shape:
        raise ValueError("s, theta, and zeta must have the same shape")

    npoints = s.size

    # Validate that arrays are not empty
    if npoints == 0:
        raise ValueError("Input arrays cannot be empty")

    points = np.zeros((npoints, 3))
    points[:, 0] = s.flatten()
    points[:, 1] = theta.flatten()
    points[:, 2] = zeta.flatten()

    field.set_points(points)

    R = field.R()[:, 0]
    Z = field.Z()[:, 0]
    nu = field.nu()[:, 0]
    phi = zeta - nu

    # Return scalars for scalar inputs, arrays for array inputs
    if input_scalar:
        return R[0], phi[0], Z[0]
    else:
        return R, phi, Z

In [4]:
boozmn_filename = "booz_xform/boozmn_LandremanPaul2021_QH_reactorScale_lowres_reference.nc"
bri = BoozerRadialInterpolant(boozmn_filename, order=3, no_K=True)
field = bri
# nfp = bri.nfp
# res = 96
# ns_interp = res
# ntheta_interp = res
# nzeta_interp = res

# ## Setup 3d interpolation
# field = InterpolatedBoozerField(
#     bri,
#     degree=3,
#     ns_interp=ns_interp,
#     ntheta_interp=ntheta_interp,
#     nzeta_interp=nzeta_interp,
# )

In [11]:
N = 50
traj_vmec = np.loadtxt("run_firm3d/precise_QH/trajectory_data_vmec_tol_1e-10_resolution_96_tmax_0.001_trapped.txt")
traj_booz = np.loadtxt("run_firm3d/precise_QH/trajectory_data_tol_1e-10_resolution_96_tmax_0.001_trapped.txt")
R, phi, Z = boozer_to_cylindrical(field, traj_booz[:N, 1], traj_booz[:N, 2], traj_booz[:N, 3])

In [12]:
X = R * np.cos(phi)
Y = R * np.sin(phi)

In [13]:
from desc.vmec import VMECIO
wout_filename = "booz_xform/wout_LandremanPaul2021_QH_reactorScale_lowres_reference.nc"
eq = VMECIO.load(wout_filename, L=8, M=8, N=8, profile="current")

In [14]:
import plotly.graph_objects as go

Np = 50
# 3D plots use grid NFP=1 to plot all toroidal surfaces
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig = plot_3d(eq, "|B|", alpha=0.3)
fig.add_trace(
    go.Scatter3d(
        x=X[:Np],
        y=Y[:Np],
        z=Z[:Np],
        mode="lines",
        line=dict(
            color="red",
            width=5,
            dash="solid",
        ),
        marker=dict(size=0),
        showlegend=False,
    )
)

In [16]:
np.savetxt("R_50_booz.txt", R)
np.savetxt("Z_50_booz.txt", Z)
np.savetxt("phi_50_booz.txt", phi)